In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.types import *

spark = SparkSession.builder \
.appName("df_project") \
.getOrCreate()

df = spark.read.option("header", True).csv("dirty_cafe_sales.csv")

In [2]:
df.printSchema()
df.show(5)
df.summary().show()

root
 |-- Transaction ID: string (nullable = true)
 |-- Item: string (nullable = true)
 |-- Quantity: string (nullable = true)
 |-- Price Per Unit: string (nullable = true)
 |-- Total Spent: string (nullable = true)
 |-- Payment Method: string (nullable = true)
 |-- Location: string (nullable = true)
 |-- Transaction Date: string (nullable = true)

+--------------+------+--------+--------------+-----------+--------------+--------+----------------+
|Transaction ID|  Item|Quantity|Price Per Unit|Total Spent|Payment Method|Location|Transaction Date|
+--------------+------+--------+--------------+-----------+--------------+--------+----------------+
|   TXN_1961373|Coffee|       2|           2.0|        4.0|   Credit Card|Takeaway|      2023-09-08|
|   TXN_4977031|  Cake|       4|           3.0|       12.0|          Cash|In-store|      2023-05-16|
|   TXN_4271903|Cookie|       4|           1.0|      ERROR|   Credit Card|In-store|      2023-07-19|
|   TXN_7034554| Salad|       2|           

In [3]:
# Создаем список новых имен
new_columns = [c.replace(" ", "_").lower() for c in df.columns]

# Применяем их к DataFrame
df = df.toDF(*new_columns)

print("Новые названия колонок:")
print(df.columns)

Новые названия колонок:
['transaction_id', 'item', 'quantity', 'price_per_unit', 'total_spent', 'payment_method', 'location', 'transaction_date']


In [4]:
total_rows = df.count()
print(f"Общее количество строк: {total_rows}")

unique_rows = df.distinct().count()
print(f"Количество уникальных строк: {unique_rows}")

print("Проверка на дубликаты по 'transaction_id':")
duplicate_counts_df = df.groupBy("transaction_id").count().filter(F.col("count") > 1).show()

Общее количество строк: 10000
Количество уникальных строк: 10000
Проверка на дубликаты по 'transaction_id':
+--------------+-----+
|transaction_id|count|
+--------------+-----+
+--------------+-----+



In [5]:
# Функция для замены невалидных значений
def replace_invalid(column):
    return F.when(F.col(column)\
    .isin(["UNKNOWN", "ERROR", "NULL"]), None)\
    .otherwise(F.col(column))

# Применение преобразований к существующим колонкам и приводим их к корректным типам данных
df = df.withColumn('item', replace_invalid('item'))
df = df.withColumn('quantity', replace_invalid('quantity').cast("integer"))
df = df.withColumn('price_per_unit', replace_invalid('price_per_unit').cast("double"))
df = df.withColumn('total_spent', replace_invalid('total_spent').cast("double"))
df = df.withColumn('payment_method', replace_invalid('payment_method'))
df = df.withColumn('location', replace_invalid('location'))
df = df.withColumn('transaction_date', replace_invalid('transaction_date').cast("date"))

print("Схема DataFrame после очистки и приведения типов:")
df.printSchema()

Схема DataFrame после очистки и приведения типов:
root
 |-- transaction_id: string (nullable = true)
 |-- item: string (nullable = true)
 |-- quantity: integer (nullable = true)
 |-- price_per_unit: double (nullable = true)
 |-- total_spent: double (nullable = true)
 |-- payment_method: string (nullable = true)
 |-- location: string (nullable = true)
 |-- transaction_date: date (nullable = true)



In [6]:
item_price=(df.dropna(subset=["item", "price_per_unit"])
.select("item", F.col("price_per_unit").alias("known_price_for_item"))
.distinct())

item_price.show()

+--------+--------------------+
|    item|known_price_for_item|
+--------+--------------------+
|Sandwich|                 4.0|
|Smoothie|                 4.0|
|   Juice|                 3.0|
|  Coffee|                 2.0|
|   Salad|                 5.0|
|     Tea|                 1.5|
|  Cookie|                 1.0|
|    Cake|                 3.0|
+--------+--------------------+



In [7]:
df_joined = df.join(item_price, on="item", how="left_outer")

df = df_joined.withColumn(
"price_per_unit",
F.coalesce(F.col("price_per_unit"), F.col("known_price_for_item"))
).drop("known_price_for_item") # Удаляем вспомогательную колонку

In [8]:
df = df.withColumn("total_spent", F.when(F.col("total_spent").isNull()
, F.col("quantity") * F.col("price_per_unit"))
.otherwise(F.col("total_spent")))


In [9]:
df = df.withColumn("quantity", F.when(F.col("quantity").isNull()
, F.col("total_spent") / F.col("price_per_unit"))
.otherwise(F.col("quantity")))

print("Статистика после частичного восстановления данных")
df.summary().show()


Статистика после частичного восстановления данных
+-------+----+--------------+------------------+------------------+-----------------+--------------+--------+
|summary|item|transaction_id|          quantity|    price_per_unit|      total_spent|payment_method|location|
+-------+----+--------------+------------------+------------------+-----------------+--------------+--------+
|  count|9031|         10000|              9977|              9946|             9977|          6822|    6039|
|   mean|NULL|          NULL| 3.024957402024657|2.9476674039815003|8.930139320437005|          NULL|    NULL|
| stddev|NULL|          NULL|1.4203945262334554|1.2806527560293721|6.004920689284521|          NULL|    NULL|
|    min|Cake|   TXN_1000555|               1.0|               1.0|              1.0|          Cash|In-store|
|    25%|NULL|          NULL|               2.0|               2.0|              4.0|          NULL|    NULL|
|    50%|NULL|          NULL|               3.0|               3.0|   

In [10]:
df = df.dropna(subset=["item", "quantity", "total_spent"])

print("Статистика после удаления строк с NULL:")
df.summary().show()

Статистика после удаления строк с NULL:
+-------+----+--------------+------------------+-----------------+-----------------+--------------+--------+
|summary|item|transaction_id|          quantity|   price_per_unit|      total_spent|payment_method|location|
+-------+----+--------------+------------------+-----------------+-----------------+--------------+--------+
|  count|9011|          9011|              9011|             9011|             9011|          6141|    5434|
|   mean|NULL|          NULL| 3.027632893130618| 2.94906225724115|8.943347020308511|          NULL|    NULL|
| stddev|NULL|          NULL|1.4237606098275837|1.279463832977388|6.016236158916319|          NULL|    NULL|
|    min|Cake|   TXN_1000555|               1.0|              1.0|              1.0|          Cash|In-store|
|    25%|NULL|          NULL|               2.0|              2.0|              4.0|          NULL|    NULL|
|    50%|NULL|          NULL|               3.0|              3.0|              8.0|    

-------------------------САМ-------------------------

На основе уже обработанного датафрема создайте новый столбец с номерами месяцев (от 1 до 12) соответствующим дате транзакции. Посчитайте общее количество транцакций для каждого месяца. В каком месяце совершено наибольшее количество транзакций?

In [11]:
df.show(5)

+------+--------------+--------+--------------+-----------+--------------+--------+----------------+
|  item|transaction_id|quantity|price_per_unit|total_spent|payment_method|location|transaction_date|
+------+--------------+--------+--------------+-----------+--------------+--------+----------------+
|Coffee|   TXN_1961373|     2.0|           2.0|        4.0|   Credit Card|Takeaway|      2023-09-08|
|  Cake|   TXN_4977031|     4.0|           3.0|       12.0|          Cash|In-store|      2023-05-16|
|Cookie|   TXN_4271903|     4.0|           1.0|        4.0|   Credit Card|In-store|      2023-07-19|
| Salad|   TXN_7034554|     2.0|           5.0|       10.0|          NULL|    NULL|      2023-04-27|
|Coffee|   TXN_3160411|     2.0|           2.0|        4.0|Digital Wallet|In-store|      2023-06-11|
+------+--------------+--------+--------------+-----------+--------------+--------+----------------+
only showing top 5 rows



In [12]:
item_date=(df.dropna(subset=["transaction_date"])
.select("transaction_date", F.month("transaction_date").alias("month")
          ).groupBy("month").count().orderBy(F.desc("count")).limit(1))
item_date.show()

+-----+-----+
|month|count|
+-----+-----+
|   10|  767|
+-----+-----+



In [13]:
item_max=(df.select("item", "price_per_unit", "total_spent", (F.col("price_per_unit") * F.col("total_spent")).alias("price"))
          .groupBy("item")
          .agg(F.max("price").alias("price"))
          .orderBy(F.desc("price"))
          .limit(1)
         )
item_max.show()

+-----+-----+
| item|price|
+-----+-----+
|Salad|125.0|
+-----+-----+



In [14]:
item_max=(df.dropna(subset=["transaction_date"])
              .select("item", "total_spent", "transaction_date")
              .where(F.month("transaction_date") == 1)
              .groupBy("item")
              .agg(F.sum("total_spent").alias("total_spent_sum"))
              .orderBy(F.desc("total_spent_sum"))
              #.limit(1)
         
         )
item_max.show()

+--------+---------------+
|    item|total_spent_sum|
+--------+---------------+
|   Salad|         1575.0|
|Sandwich|         1384.0|
|   Juice|          918.0|
|    Cake|          855.0|
|Smoothie|          756.0|
|  Coffee|          556.0|
|     Tea|          396.0|
|  Cookie|          250.0|
+--------+---------------+



In [15]:
item_max = (df.dropna(subset=["transaction_date"])
              .select("item", "total_spent", "transaction_date")
              .where(F.month("transaction_date") == 1)
              .groupBy("item", F.month("transaction_date"))  # Добавляем дату в группировку
              .agg(F.sum("total_spent").alias("total_spent_sum"))
              .orderBy(F.desc("total_spent_sum"))
         )
item_max.show()

+--------+-----------------------+---------------+
|    item|month(transaction_date)|total_spent_sum|
+--------+-----------------------+---------------+
|   Salad|                      1|         1575.0|
|Sandwich|                      1|         1384.0|
|   Juice|                      1|          918.0|
|    Cake|                      1|          855.0|
|Smoothie|                      1|          756.0|
|  Coffee|                      1|          556.0|
|     Tea|                      1|          396.0|
|  Cookie|                      1|          250.0|
+--------+-----------------------+---------------+



In [16]:
item_max=df.select(F.round(F.avg("total_spent"), 2).alias("avg_rounded"))
         #df.select(F.round(F.avg("total_spent"), 2).alias("avg_rounded"))
item_max.show()

+-----------+
|avg_rounded|
+-----------+
|       8.94|
+-----------+



In [22]:
item_max = (df.dropna(subset=["location", "payment_method"])
            .select(df.location, df.payment_method)
            .where((df.location == "In-store") & (df.payment_method == "Credit Card"))
            .count()
           )

item_max

611